## 31 - Radar Animations

Watch video: [YouTube](https://www.youtube.com/watch?v=4paJWaQkUF8)

In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from metpy.plots import ctables
from siphon.cdmr import Dataset
from siphon.catalog import TDSCatalog
from siphon.radarserver import RadarServer
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
cat = TDSCatalog('http://thredds.ucar.edu/thredds/radarServer/catalog.xml')
url = cat.catalog_refs['NEXRAD Level III Radar from IDD'].href
rs = RadarServer(url)
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = datetime.datetime.utcnow()
query = rs.query()
query.stations('NQA')
query.time_range(start, end)
query.variables('N0C')

var=N0C&time_start=2024-03-25T02%3A11%3A42.158050&time_end=2024-03-25T03%3A11%3A42.158050&stn=NQA

In [3]:
query_cat = rs.get_catalog(query)
sorted(list(query_cat.datasets))

['Level3_NQA_N0C_20240325_0214.nids',
 'Level3_NQA_N0C_20240325_0217.nids',
 'Level3_NQA_N0C_20240325_0222.nids',
 'Level3_NQA_N0C_20240325_0226.nids',
 'Level3_NQA_N0C_20240325_0231.nids',
 'Level3_NQA_N0C_20240325_0235.nids',
 'Level3_NQA_N0C_20240325_0238.nids',
 'Level3_NQA_N0C_20240325_0241.nids',
 'Level3_NQA_N0C_20240325_0245.nids',
 'Level3_NQA_N0C_20240325_0248.nids',
 'Level3_NQA_N0C_20240325_0252.nids',
 'Level3_NQA_N0C_20240325_0255.nids',
 'Level3_NQA_N0C_20240325_0258.nids',
 'Level3_NQA_N0C_20240325_0301.nids',
 'Level3_NQA_N0C_20240325_0304.nids',
 'Level3_NQA_N0C_20240325_0307.nids']

In [4]:
def plot_radar(data, field_name: str):
 rng = data.variables['gate'][:]
 az = data.variables['azimuth'][:]
 ref = data.variables[field_name][:]
 distance_in_degrees = 3
 ax.set_extent([data.RadarLongitude - distance_in_degrees, data.RadarLongitude + distance_in_degrees, data.RadarLatitude - distance_in_degrees, data.RadarLatitude + distance_in_degrees])
 x = rng * np.sin(np.deg2rad(az))[:, np.newaxis]
 y = rng * np.cos(np.deg2rad(az))[:, np.newaxis]
 norm, cmap = ctables.registry.get_with_range('Carbone42', 0, 1)
 mesh = ax.contour(x, y, ref, norm=norm, cmap=cmap, zorder=0)
 return mesh

In [5]:
field_name = 'CorrelationCoefficient'
base_file = query_cat.datasets[0].remote_access()
list(base_file.variables)

['elevation',
 'azimuth',
 'gate',
 'latitude',
 'longitude',
 'altitude',
 'rays_time',
 'CorrelationCoefficient_RAW',
 'CorrelationCoefficient']

In [7]:
proj = ccrs.LambertConformal(central_longitude=base_file.RadarLongitude, central_latitude=base_file.RadarLatitude)

In [ ]:
# This takes lots of time to execute...
fig = plt.figure(figsize=(6, 8))
ax = fig.add_subplot(projection=proj)
artist = []
ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=1, zorder=2)

for ds_name in sorted(query_cat.datasets):
    ds = query_cat.datasets[ds_name]
    data = Dataset(ds.access_urls['CdmRemote'])
    field_name = [var.name for var in data.variables.values() if var.ndim >= 2 and not var.name.endswith('RAW')][0]
    text = ax.text(0.5, 1.02, data.time_coverage_start, ha='center', transform=ax.transAxes)
    mesh = plot_radar(data, field_name)
    artist.append([text, mesh])

In [ ]:
from matplotlib.animation import ArtistAnimation
plt.rcParams['animation.html'] = 'jshtml'
anim = ArtistAnimation(fig, artist, interval=100)
anim